# SumLinks para argentina
La idea es usar el el raw CDRs dataset para crear el equivalente al sumlinks de mexico que cuenta agrega pares de usuarios y resume informacion de comunicacion entre los 2 usuarios, asi como la distincion de la direccion de uso.

El output tendria que tener el siguiente header:
USER|OTHER_USER|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend


## definiciones utilizadas

* la fecha viene dado en YYYMMDD y el TIME es HHMMSS en formato 24 horas
* *Time* columns are measured in seconds.
* Daylight corresponds to the [8 20) timeframe (using Arg's timezone)
* el analisis es solo para CDRs con usuarios de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming ,1, si entra y viceversa si es outgoing,0).

### el dataset crudo vendria con este header
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|

CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO

|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF

Basicamente los primeros 2 son los hashes de los users que estuvieron en el call, con el primero el user de la telco.

Direction es quien llama a quien (outgoing es 1-->2, viceversa incoming). Fecha el dia,mes,anyo del llamado, time en formato 24hs incluyendo segundos. Duration medido en segundos, Client_cell_id el hash de la antenna y todo el resto no nos interesa.

### la idea es llegar a algo similar a esto
pero sin la info de sms

In [1]:
!zcat /grandata/voice/sum_links/sum_links_201401_201403.txt.gz | head

LineKeyOrigin|LineKeyTarget|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend|SmsWeekDaylight|SmsWeekNight|SmsWeekend
BA3439E244F09BFA85F03750D84DB1B7|5AA26ADB35AF1A26A635B7747320732F|1|0|0|44|0|0|0|0|0
BA3439E244F09BFA85F03750D84DB1B7|503D0DC95765D42BE5CFB3C50075214A|1|1|0|78|313|0|0|0|0
BA3439E244F09BFA85F03750D84DB1B7|EA6C583D0F31F6D2A1E23B55334DF494|0|0|2|0|0|98|0|0|0
73F169CBB3B6E443B896017C51B1C600|9D0AFC1877F147C63E235CF25B42CC0B|1|4|4|6|85|55|0|0|0
7A0B5B991DD1B3C6E1498C07EF02FD31|9D0AFC1877F147C63E235CF25B42CC0B|0|2|5|0|146|268|0|0|0
EA23ADE2371B8D5FB02A8DDBAF5A53B2|9D0AFC1877F147C63E235CF25B42CC0B|2|1|0|147|46|0|0|0|0
BC834644645F540C1BD72307A5BD5B96|9D0AFC1877F147C63E235CF25B42CC0B|0|0|1|0|0|9|0|0|0
3C43E7D3B99947C9C0C21BA7D387F26A|9D0AFC1877F147C63E235CF25B42CC0B|1|2|0|7|142|0|0|0|0
97B89BED4B7123F6F6F47B41403C4DF1|9D0AFC1877F147C63E235CF25B42CC0B|0|1|0|0|48|0|0|0|0

gzip: stdout: Broken pipe


## Read Sframes

In [2]:
import pandas as pd;
import numpy as np;
import os;
import random;
import graphlab as gl
import time
import datetime
import sys
import subprocess
pd.set_option('display.max_rows', 200)
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')
rootdir="/home/juan/mobility-study/argentina-scripts"
os.chdir(rootdir)

#los meses a analizar
ms = [11,12,1,2,3]

def get_output_file(month , group = -1,sample=False):
    output = "/home/juan/mobility-study/argentina-scripts/output/2012/{0}/sum_links_".format(month,group)
    if sample == True:
        output = output + "_sample"
    if group != -1:
        output = output + "_group{g}_1".format(g=group)
    return output



/home/juan/mobility-study/scikit-learn/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 38] Function not implemented.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [3]:
!head "/home/juan/mobility-study/argentina-scripts/data/celdas_dropped.csv"

SF103
SF106
SF960
TEBBLA
RG086
PPVEN
CH022
SC999
RG037
ST999


In [4]:
!wc -l "/home/juan/mobility-study/argentina-scripts/data/celdas_limpio.csv"

5807 /home/juan/mobility-study/argentina-scripts/data/celdas_limpio.csv


In [5]:
antennas_file = '/home/juan/mobility-study/argentina-scripts/data/celdas_limpio.csv'
antennas = gl.SFrame.read_csv( antennas_file, delimiter='|', 
                                     header=True, skip_initial_space=True, 
                                         column_type_hints = [str,float, float, str, str, bool], 
                                         na_values=['NaN'],
                                         usecols = ['CEL_ID','LATITUD',
                                                    'LONGITUD','DEPARTAMENTO','PROVINCIA','EPIDEMIC'],  
                                         error_bad_lines=False,
                                         verbose = False
                                        )                

2016-07-02 16:06:42,710 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1467475601.log


This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [6]:
month = 2
#check february days length
if month ==2:
    days = range(1,30)

days = range(1,8)


## First test run

In [11]:
year = 2012
day = 11
sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)        
daily_table = gl.load_sframe(sframe_dir)

In [12]:
daily_table.head()

USER,OTHER_USER,DIRECTION,DATE,TIME,CLIENT_CELL_ID
703344566D793559654176393030796754677A4F45513D3D ...,3861696F2B5634476F63466873656E6B485361634B413D3D ...,O,20120211,205352,CF174Z
703344566D793559654176393030796754677A4F45513D3D ...,3861696F2B5634476F63466873656E6B485361634B413D3D ...,O,20120211,144748,C1541K
41452B6F48655752536F6F6342436F79542B666878413D3D ...,557765712B496D346330445772734733462B73504C773D3D ...,O,20120211,140500,CTI 1
654A50395371384250416B32725545694131626744673D3D ...,37736F4C4652757232523944564A793963354E5338673D3D ...,O,20120211,222320,CTI 1
654A50395371384250416B32725545694131626744673D3D ...,37736F4C4652757232523944564A793963354E5338673D3D ...,O,20120211,222347,CTI 1
654A50395371384250416B32725545694131626744673D3D ...,37736F4C4652757232523944564A793963354E5338673D3D ...,O,20120211,222420,CTI 1
4B484B30306C69324E436A4E476859584A49344974773D3D ...,656A6B6C45545358686F397278447578634A6B6D54513D3D ...,O,20120211,224705,CTI 1
323368576D6A6D312F6768416C6E2F423866504F72773D3D ...,325035363533493943784F584E75666B4650534170513D3D ...,O,20120211,163820,CTI 1
77596F514B33647845415235476A5079792F4B536D673D3D ...,72465075516667395678552B343768456962616B76413D3D ...,O,20120211,220740,CTI 1
3273707045584B626F50354769684B527075457168413D3D ...,36756D4C78775872577466544D384170734F31594A773D3D ...,O,20120211,204934,CTI 1


In [13]:
%%time

## script para extraer todos los atributos del simple_format
#ver el tiempo que tarda
start_time = time.time()

       
#output_file = get_output_file(month,group)
#agrego este control para cuando tuve que cortar antes el algo por alguna razon
#if (os.path.exists(output_file)):
#    print('skipping group {gr} since output-dir exists: {di}'.format(gr = group, di =output_file))

    
#group = 0
#passes = 20
#local_
# print("working group number {it} of {pas}, time elapsed is {t}, localtime is {ho}:{mi} \n".\
#    format(it=group,pas=passes, t=(time.time()-start_time),ho=local_time.tm_hour,mi=local_time.tm_min))
#itero sobre los meses
table =  gl.SFrame()
for day in days:

    sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)        
    daily_table = gl.load_sframe(sframe_dir)
    
    #a cada tabla diaria filtro por todos los USERs por el hash del string (el hash es un int) y despue filtro modulo passes
    #y trabajo sobre la tabla subgroup nada mas que ahora tiene menos usuarios
    #table = table.append(daily_table[daily_table['USER'].apply(lambda x: hash(x) % passes == group )])
    table = table.append(daily_table.head(10**6))
    
#entonces la idea es que yo ahora solo voy a trabajar, dentro de esta tabla filtrada y para todos los dias del mes juntos


CPU times: user 26.6 s, sys: 888 ms, total: 27.5 s
Wall time: 39.5 s


In [14]:
#table2 = table.copy()
#table =  table2.copy()

In [15]:
%%time
#paso las fechas a un timestamp
table['TIMESTAMP'] = table['DATE','TIME'].apply(lambda x: x['DATE'][0:4]+'-'+x['DATE'][4:6]+'-'+x['DATE'][6:8]
                                 +'-'+x['TIME'][0:2]+'-'+x['TIME'][2:4]+'-'+x['TIME'][4:6])

table['TIMESTAMP'] = table['TIMESTAMP'].str_to_datetime(str_format='%Y-%m-%d-%H:%M:%S')

## leo si las llamadas fueron hechas durante el findesemana
table['DURING_WEEKEND'] = table['TIMESTAMP'].apply(lambda x: x.weekday()==5 or x.weekday()==6)

# asigno el bool si la llamada fue en horario 'laboral'
#notar que la hora < 19 significa que 18hs y 59 min da True
table['DURING_DAYLIGHT']= table['TIMESTAMP'].apply(lambda x: x.hour>=8 and x.hour<20 )

CPU times: user 56 s, sys: 5.08 s, total: 1min 1s
Wall time: 27.5 s


In [20]:
table['DURING_WEEKEND'].show()

In [21]:
table['DURING_DAYLIGHT'].show()

CallsWeekDaylight | CallsWeekNight | CallsWeekend | TimeWeekDaylight | TimeWeekNight | TimeWeekend


In [22]:
table_weeknight = table[(table['DURING_DAYLIGHT']==False) &  (table['DURING_WEEKEND']==False)]

In [23]:
table_weeknight.shape, table.shape

((1385694, 11), (7000000, 11))

In [24]:
%%time
table['DURING_DAYLIGHT']= table['TIMESTAMP'].apply(lambda x: x.hour>=8 and x.hour<20 )

table_weeknight = table[(table['DURING_DAYLIGHT']==False) &  (table['DURING_WEEKEND']==False)]

table_weekend = table[table['DURING_WEEKEND']==True]

#table_weeknight = table[(table['DURING_DAYLIGHT']==False) &  (table['DURING_WEEKEND']==False)]

table_weekday = table[(table['DURING_DAYLIGHT']==True) &  (table['DURING_WEEKEND']==False)]


CPU times: user 48 ms, sys: 20 ms, total: 68 ms
Wall time: 66.2 ms


In [25]:
%%time
 table_weekend_call = table_weekend.groupby(['USER','OTHER_USER','DIRECTION'],
                             {'CallsWeekEnd':gl.aggregate.COUNT('TIMESTAMP')})

table_weeknight_call = table_weeknight.groupby(['USER','OTHER_USER','DIRECTION'],
                 {'CallsWeekNight':gl.aggregate.COUNT('TIMESTAMP')})


table_weekday_call = table_weekday.groupby(['USER','OTHER_USER','DIRECTION'],
                 {'CallsWeekDay':gl.aggregate.COUNT('TIMESTAMP')})


table_weekend_time = table_weekend.groupby(['USER','OTHER_USER','DIRECTION'],
                 {'TimeWeekEnd':gl.aggregate.SUM('DURATION')})


table_weeknight_time = table_weeknight.groupby(['USER','OTHER_USER','DIRECTION'],
                 {'TimeWeekNight':gl.aggregate.SUM('DURATION')})


table_weekday_time = table_weekday.groupby(['USER','OTHER_USER','DIRECTION'],
                 {'TimeWeekDay':gl.aggregate.SUM('DURATION')})

CPU times: user 2min 56s, sys: 19.4 s, total: 3min 15s
Wall time: 1min 48s


In [26]:
# ahora mergeamos todas las tablas anteriores con un outer join en c/ casi Y FILLNA
#recordar que para graphlab el fillna es por columna!

In [27]:
table = table_weekend_call.join(table_weeknight_call,on = ['USER','OTHER_USER','DIRECTION'], how = 'outer')
del table_weekend_call, table_weeknight_call

table =  table.join(table_weekday_call,on = ['USER','OTHER_USER','DIRECTION'], how = 'outer')
del table_weekday_call

table =  table.join(table_weekend_time,on = ['USER','OTHER_USER','DIRECTION'], how = 'outer')
del table_weekend_time

table =  table.join(table_weekday_time,on = ['USER','OTHER_USER','DIRECTION'], how = 'outer')
del table_weekday_time

table =  table.join(table_weeknight_time,on = ['USER','OTHER_USER','DIRECTION'], how = 'outer')
del table_weeknight_time



del table_weekend, table_weeknight, table_weekday

In [28]:
for col in table.column_names():
                if ('USER' in col) or ('DIRECTION' in col):
                    continue
                table = table.fillna(col,0)

In [34]:
table.sample(1e-5)

DIRECTION,OTHER_USER,USER,CallsWeekEnd,CallsWeekNight,CallsWeekDay
O,755A534B4C5032772B7658644341414F784D745552673D3D ...,6F3674736A315955514634765033725557562B4F34513D3D ...,0,0,1
O,372F7145354C636944622B5A51426C57352F714456773D3D ...,4B733773373179365A6E424C474953557963385846773D3D ...,0,0,1
O,4D4232304A757342344A49316348586C634E543570673D3D ...,716E6F78763036673471414D782F2B677A59753662413D3D ...,0,0,4
O,4A7358464E6F4C6F366455494D33763970556A5571673D3D ...,34454157716F445831797443563045426F5870366E513D3D ...,0,0,1
O,4352594E664A42704D6D6141434F43496D312F7557513D3D ...,4634636949743762766B373679374558376F304C73773D3D ...,0,0,1
O,733436447A46435A4746354B547739595031575041673D3D ...,6856626363354853594677372F7A36455273525357773D3D ...,0,0,1
O,764D54397146433233455669556565695174347053513D3D ...,61366450486D396F63767779396B79545054675974513D3D ...,0,0,1
O,474F3039766739452B36474B684D4F50484A497059773D3D ...,42486A36345A71484F32462B397A31736864357842773D3D ...,0,0,1
O,4F79447570656A6A366F57735530514B61446B5738773D3D ...,374C6E6333682B376A565638434F2B334762346E4A413D3D ...,0,0,13
O,4944424E42774162433676335362736179524E3559413D3D ...,4F4C6675544E2F5049717A67616E2B317A32422F6C773D3D ...,0,0,1


In [31]:
gl.canvas.set_target('ipynb')


In [32]:
table['CallsWeekDay'].show()

In [32]:
table['CallsWeekNight'].show()

In [33]:
table['CallsWeekEnd'].show()

In [34]:
table['TimeWeekDay'].show()

In [35]:
table['TimeWeekNight'].show()

In [36]:
table['TimeWeekEnd'].show()

## consitency check from table2
* check we're not losing unique num of user/other_users
* check random user samples

In [7]:
%%time
total_users_in = len(table2.groupby('USER',
                 {'count':gl.aggregate.COUNT('DURATION')}))
total_users_out = len(table.groupby('USER',
                 {'count':gl.aggregate.COUNT('CallsWeekNight')}))
print(total_users_in,total_users_out)

NameError: name 'table2' is not defined

In [8]:
%%time
total_ousers_in = len(table2.groupby('OTHER_USER',
                 {'count':gl.aggregate.COUNT('DURATION')}))
total_ousers_out = len(table.groupby('OTHER_USER',
                 {'count':gl.aggregate.COUNT('CallsWeekNight')}))
print(total_ousers_in,total_ousers_out)

NameError: name 'table2' is not defined

In [88]:
table2['USER'].sample(0.01).head(10)[0]

'5030367459586451306D655135545635714337704C513D3D'

In [89]:
usr = "5030367459586451306D655135545635714337704C513D3D"

before = table2[table2['USER']==usr][['USER','OTHER_USER','DATE','TIME','DURATION']]

In [90]:
after = table[table['USER']==usr]

In [91]:
set(before['OTHER_USER'].unique()) == set(after['OTHER_USER'].unique())

True

In [92]:
set(before['USER'].unique()) == set(after['USER'].unique())

True

In [93]:
# wednesday 01/02 thu 02/02 fri 03/02 sat 04/02 sun 05/02 mon 06/02 tue 07/02

In [94]:
before

USER,OTHER_USER,DATE,TIME,DURATION
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,223333,476
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,225210,80
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,224653,132
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,213355,488
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,224245,144
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,222837,44
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,162914,132
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,153749,3011
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,153605,1
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120203,215933,1


In [95]:
# wednesday 01/02 thu 02/02 fri 03/02 sat 04/02 sun 05/02 mon 06/02 tue 07/02

In [96]:
after

Columns:
	OTHER_USER	str
	USER	str
	CallsWeekEnd	int
	CallsWeekNight	int
	CallsWeekDay	int
	TimeWeekEnd	int
	TimeWeekDay	int
	TimeWeekNight	int

Rows: Unknown

Data:
+-------------------------------+-------------------------------+--------------+
|           OTHER_USER          |              USER             | CallsWeekEnd |
+-------------------------------+-------------------------------+--------------+
| 474A6D773639717256495A6658... | 5030367459586451306D655135... |      1       |
+-------------------------------+-------------------------------+--------------+
+----------------+--------------+-------------+-------------+---------------+
| CallsWeekNight | CallsWeekDay | TimeWeekEnd | TimeWeekDay | TimeWeekNight |
+----------------+--------------+-------------+-------------+---------------+
|       8        |      3       |      2      |     3144    |      1367     |
+----------------+--------------+-------------+-------------+---------------+
[? rows x 8 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.

In [38]:
group =3
get_output_file(month,group)

'/home/juan/mobility-study/argentina-scripts/output/2012/2/sum_links__group3_1'

# cambiamos el input/output_sframe
(output/2012/2/sum_links__group3_1  --> output/2012/2/sum_links_2)

Esto pues vamos salvando los resultados del procesamiento de datos en files sucesivos para no tener que repetir el algoritmo ni para arriesgar cambiar una version (en terminos de datos) y no tener checkpoints intermedios

## Ahora
La idea es colapsar la columna de direction, filtrar los users a los cuales les podemos dar un home_antenna, agregar la info per_antenna.

## Esta parte sera manual

La idea es ir controlando los datos, los input/output dirs, etc. a manos en el procesamiento

In [7]:
get_input_file = get_output_file

In [8]:
def get_output_file2(month, g1,g2 , direction = 'O', sample=False):
    if not(isinstance( month, int ) and isinstance( g1, int ) and isinstance( g2, int )):
        print('first group, last group or month is not int type')
        return
    if not(direction in ['O','I']):
        print('direction can only O or I chars')
        return
    output = "/home/juan/mobility-study/argentina-scripts/output/2012/{m}/sum_links2_g{g1}tog{g2}_direc{d}"\
                    .format(m=month,g1=g1,g2=g2,d =  direction)
    if sample == True:
        output = output + "_sample"
    return output


In [9]:
get_output_file2(1,1,5,'O',True)

'/home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links2_g1tog5_direcO_sample'

## Cargar el home antenna mapping p/c/ user

In [11]:
homeantenna_dirs = [x[0] for x in os.walk('output/') if ("homeantenna" in x[0] and not("sample" in x[0]))]
homeantenna_dirs

['output/homeantenna_sframe_group1_1',
 'output/homeantenna_sframe_group17_1',
 'output/homeantenna_sframe_group2_1',
 'output/homeantenna_sframe_group8_1',
 'output/homeantenna_sframe_group19_1',
 'output/homeantenna_sframe_group0_1',
 'output/homeantenna_sframe_group16_1',
 'output/homeantenna_sframe_group11_1',
 'output/homeantenna_sframe_group10_1',
 'output/homeantenna_sframe_group6_1',
 'output/homeantenna_sframe_group14_1',
 'output/homeantenna_sframe_group12_1',
 'output/homeantenna_sframe_group9_1',
 'output/homeantenna_sframe_group3_1',
 'output/homeantenna_sframe_group4_1',
 'output/homeantenna_sframe_group7_1',
 'output/homeantenna_sframe_group5_1',
 'output/homeantenna_sframe_group15_1',
 'output/homeantenna_sframe_group13_1',
 'output/homeantenna_sframe_group18_1']

In [12]:
%%time
##read homeantenna_data
homeantenna_map =  gl.SFrame()

for home_dir in homeantenna_dirs:
    group_map = gl.load_sframe(home_dir)
    group_map = group_map['USER','ANTENNA_ID_WEEKNIGHT_0', 'ANTENNA_ID_0']
    #users with str(-1) values are those for which no calls were recorded in that period, thus we have to discard them
    group_map = group_map[group_map['ANTENNA_ID_WEEKNIGHT_0']!='-1']
    homeantenna_map = homeantenna_map.append(group_map)

print(homeantenna_map.shape)


(31475183, 3)
CPU times: user 2min 37s, sys: 26.4 s, total: 3min 3s
Wall time: 1min 20s


In [11]:
##  check how antenna_id_0 and antenna_id_weeknight_0 filters differ
#(homeantenna_map['ANTENNA_ID_WEEKNIGHT_0']==homeantenna_map['ANTENNA_ID_0']).sum()

In [12]:
## sanity check its all unique users in the mapping
#len(homeantenna_map['USER'].unique())

In [13]:
#keep caro's filter
homeantenna_map = homeantenna_map['USER', 'ANTENNA_ID_WEEKNIGHT_0']

In [14]:
homeantenna_map.head()

USER,ANTENNA_ID_WEEKNIGHT_0
4C6862586458547A313548437A74793733685A5737413D3D ...,BA237
742F364F65466756413236674E6B4449766369646D673D3D ...,C1664
587442774E45766C733553526C376335456566566D413D3D ...,SF449
44713446416D556448624A4B6B306E416452643655673D3D ...,ME068
6552796530623052497A6A75724C514B7276563576673D3D ...,CO670
37556C7676356A38707036774C4A44373335325555773D3D ...,C2366
3444617664304C5676444B41373152372F49754946513D3D ...,BA820
714652715A4B64544C32695371614F56355047364A773D3D ...,CF001
4761514E637346704F31617877506F4B494F33682B773D3D ...,JU200
6154474F55662F4656444A68377A726F424D4A3043773D3D ...,NQ003


In [15]:
group =2
input_dir =  get_input_file(month = 1,group=group,sample=False)
!du -sh $input_dir
!ls output/2012/1/

1018M	/home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group2_1
sum_links2_g0tog4_dirO	    sum_links__group14_1  sum_links__group3_1
sum_links2_g14tog19_direcO  sum_links__group15_1  sum_links__group4_1
sum_links2_g5tog13_direcO   sum_links__group16_1  sum_links__group5_1
sum_links__group0_1	    sum_links__group17_1  sum_links__group6_1
sum_links__group10_1	    sum_links__group18_1  sum_links__group7_1
sum_links__group11_1	    sum_links__group19_1  sum_links__group8_1
sum_links__group12_1	    sum_links__group1_1   sum_links__group9_1
sum_links__group13_1	    sum_links__group2_1


# OBS IMPORTANTE
para comparar con caro solo voy a hacer llamados salientes pues esto me reduce el tamanyo del dataset y ademas porque la mayoria de los mapas visualizados en Processing son de _out_


# param setting

In [10]:
month = 1
#grupo a analizar desde
g1 = 14
#grupo a analizar hasta
g2 = 19
groups = range(g1,g2+1)
direction = 'O'
output_dir = get_output_file2(month,g1,g2,direction)
print(output_dir)
!ls output/2012/1/
month,g1,g2,direction

/home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links2_g14tog19_direcO
antenna_usage_aggregation.csv  sum_links__group13_1  sum_links__group2_1
sum_links2_g0tog4_direcO       sum_links__group14_1  sum_links__group3_1
sum_links2_g14tog19_direcO     sum_links__group15_1  sum_links__group4_1
sum_links2_g5tog13_direcO      sum_links__group16_1  sum_links__group5_1
sum_links__group0_1	       sum_links__group17_1  sum_links__group6_1
sum_links__group10_1	       sum_links__group18_1  sum_links__group7_1
sum_links__group11_1	       sum_links__group19_1  sum_links__group8_1
sum_links__group12_1	       sum_links__group1_1   sum_links__group9_1


(1, 14, 19, 'O')

In [18]:
get_input_file(month,group,False)

'/home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group2_1'

In [19]:
%%time
table =  gl.SFrame()

for group in groups:
    sframe_dir = get_input_file(month,group,False)
    print('loading sframe dir %s' % sframe_dir)
    group_table = gl.load_sframe(sframe_dir)
    group_table = group_table[group_table['DIRECTION']=='O']
    group_table.remove_column('DIRECTION')
    #nos quedamos con solo aquellos users/other_users de los cuales sabemos su home_antenna
    #group_table.filter_by(homeantenna_map['USER'], 'USER')
    #group_table.filter_by(homeantenna_map['USER'], 'OTHER_USER')
    table = table.append(group_table)

del group_table
print(table.shape)

loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group14_1
loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group15_1
loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group16_1
loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group17_1
loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group18_1
loading sframe dir /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links__group19_1
(39098453, 8)
CPU times: user 5min 21s, sys: 1min 8s, total: 6min 30s
Wall time: 1min 50s


In [ ]:
#%%time
#print(group_table.shape)
#group_table.filter_by(homeantenna_map['USER'], 'USER')
#print(group_table.shape)

## levanto
de 3GB a 14GB la memoria ram y tardo 337s

In [19]:
#%%time
#print(table.shape)
#table.filter_by(homeantenna_map['USER'], 'OTHER_USER')
#print(table.shape)

(13558605, 9)
(13558605, 9)
CPU times: user 12min 50s, sys: 38.2 s, total: 13min 28s
Wall time: 4min 44s


## levanto extra
de 14GB a la memoria ram y tardo 303s

In [94]:
#%%time
#cambio de lado el user, el other_user de acuerdo a la direccion del llamado y colapso la info
#group_table['NEW_USER'] = group_table['DIRECTION','OTHER_USER', 'USER' ].apply(lambda x: x['OTHER_USER'] if x['DIRECTION'] == 'I' \
#                                                            else x['USER'])
#group_table['NEW_OTHER_USER'] = group_table['DIRECTION','OTHER_USER', 'USER' ].apply(lambda x: x['USER'] if x['DIRECTION'] == 'I' \
#                                                            else x['OTHER_USER'])

In [20]:
table.sample(0.00001)['USER' ,'OTHER_USER' ]

USER,OTHER_USER
58713458332F6E61596E756862372F4C496E614A74513D3D ...,2B30713765483758526748415836616679464F504C773D3D ...
2B695550445630666B4F5443504B4752766F4D5A67413D3D ...,676F514A6D53703055455733573051435541622F43773D3D ...
4F52526332422F41494C666E4F4C68654656304B78413D3D ...,6C627A7171543551734176614D374A367072334D57773D3D ...
417339594D6F7A646E4E49524D76454E416E546733513D3D ...,5772764A4B694E446E34554B515054353935793161513D3D ...
357954365274764D766A7554342B547A5153314259513D3D ...,466D31722B575A78356C306E6364445A484274514B673D3D ...
6777774863456D5052733367714855352F34305873673D3D ...,454D632B754A53686F3838442B64306D573642686B673D3D ...
4D784474522F664432746E482F78484C3663762F6A513D3D ...,33546F5A6A6B647048646B3933317161392F566F51413D3D ...
6F6C456E752F77355071445454744D744E50546648413D3D ...,576D5A446C77617642746358316B53465266596373773D3D ...
4E4D3762316D70584545587465696831664F624345673D3D ...,6D71467A3870345251444373616E7046706C6B4176773D3D ...
3648352F2F416242523367515774715A652B2B6851513D3D ...,617834676843795A4E4262726F6E6A316D654A7A36513D3D ...


# en este caso
solo utilizamos el filtro horario correspondiente p/c/ antenna

In [ ]:
#mergeo con la info de epidemicidad y me quedo solo si un usuario es o no epidemico.
homeantenna_map = homeantenna_map.join(antennas['CEL_ID', 'EPIDEMIC'], on ={'ANTENNA_ID_WEEKNIGHT_0':'CEL_ID' },how='left')

homeantenna_map.remove_columns(['ANTENNA_ID_WEEKNIGHT_0'])

USER,EPIDEMIC
4C6862586458547A313548437A74793733685A5737413D3D ...,0
742F364F65466756413236674E6B4449766369646D673D3D ...,0
587442774E45766C733553526C376335456566566D413D3D ...,0
44713446416D556448624A4B6B306E416452643655673D3D ...,0
6552796530623052497A6A75724C514B7276563576673D3D ...,0
37556C7676356A38707036774C4A44373335325555773D3D ...,0
3444617664304C5676444B41373152372F49754946513D3D ...,0
714652715A4B64544C32695371614F56355047364A773D3D ...,0
4761514E637346704F31617877506F4B494F33682B773D3D ...,0
6154474F55662F4656444A68377A726F424D4A3043773D3D ...,0


## Definiciones
Un user es vulnerable sii tiene al menos 1 target_user (importante tomar en cuenta la direction del call) viviendo en una zona epidemica

Un user es epidemico sii su antenna esta en la zona del GC

In [32]:
#%%time
#count_nulls = homeantenna_map[ homeantenna_map['EPIDEMIC'] == None ]
#print(count_nulls.shape)

(0, 2)
CPU times: user 40.2 s, sys: 9.74 s, total: 50 s
Wall time: 5.7 s


In [ ]:
%%time
table = table.join(homeantenna_map, on = {'OTHER_USER':'USER'} ,how='left') # el on = dict() es una especie de left_on, right_on
print(table.shape)

In [ ]:
%%time
## Los que tienen epidemic == None los tenemos que tirar pues son de other_users que no sabemos cual es su home_antenna
count_nulls = table[ table['EPIDEMIC'] == None ]
print(count_nulls.shape)

In [ ]:
table = table[ table['EPIDEMIC'] != None ]

# logs tomando groups de a 5
* January, direction O: num sumlinks is 32331019 and num_nulls is 6402941, 8 groups with 58470712 pairs and (11708613, 9) nulls
*


In [ ]:
#%%time
#table.save(output_dir)

In [ ]:
#table = gl.load_sframe(output_dir)

In [ ]:
#!du -ha $output_dir

In [ ]:
#table.head()

In [ ]:
table_epi = table[table['EPIDEMIC']==1]
table = table[table['EPIDEMIC']==0]

In [ ]:
%%time
table_epi = table_epi.groupby('USER',
                {'CallsWeekEnd':gl.aggregate.SUM('CallsWeekEnd'),
                 'CallsWeekNight':gl.aggregate.SUM('CallsWeekNight'),
                'CallsWeekDay':gl.aggregate.SUM('CallsWeekDay'),
                'TimeWeekEnd':gl.aggregate.SUM('TimeWeekEnd'),
                'TimeWeekNight':gl.aggregate.SUM('TimeWeekNight'),
                'TimeWeekDay':gl.aggregate.SUM('TimeWeekDay')}
                             )


In [ ]:
rename_dict =  dict([(col,col+"_EPI") if not('USER' in col) else (col,col) for col in table_epi.column_names() ])
rename_dict
#rename_dict['VULNERABLE'] = 'VULNERABLE'

In [ ]:
table_epi.rename(rename_dict)

In [ ]:
table_epi['VULNERABLE'] = 1

In [ ]:
%%time
table = table.groupby('USER',
                {'CallsWeekEnd':gl.aggregate.SUM('CallsWeekEnd'),
                 'CallsWeekNight':gl.aggregate.SUM('CallsWeekNight'),
                'CallsWeekDay':gl.aggregate.SUM('CallsWeekDay'),
                'TimeWeekEnd':gl.aggregate.SUM('TimeWeekEnd'),
                'TimeWeekNight':gl.aggregate.SUM('TimeWeekNight'),
                'TimeWeekDay':gl.aggregate.SUM('TimeWeekDay')}
                             )
print(table.shape)

In [ ]:
table = table.join(table_epi, on = 'USER' ,how='outer')
print(table.shape)

In [43]:
table.head()

USER,TimeWeekEnd,CallsWeekDay,CallsWeekNight,TimeWeekDay,CallsWeekEnd,TimeWeekNight
36766C5262716643446A53514F7246534E63664278673D3D ...,653,149,38,7470,23,1476
2F63486368745777513669346F364B363732694349673D3D ...,530,14,9,461,12,383
655A7341594B59396A5942617937645044714E7559513D3D ...,292,6,9,199,7,237
4F4A4A6D65304F3471544E7174354D464941707948773D3D ...,0,1,0,49,0,0
4345776E53674439427A3639342F54666B66693530673D3D ...,171,11,3,445,6,151
35584A656A396B516F7337365163326A62412B5A6D413D3D ...,9,11,2,64,2,8
4B494D415637543555316F307A324D42427976442B673D3D ...,943,50,12,6078,21,547
36367A4D7846664331413462396B5A686235704449773D3D ...,0,28,46,44,0,572
453436512F4C3338697031624876634A794F393472513D3D ...,1752,54,20,2813,27,1358
7976525842754576683568475763706B7845364D5A673D3D ...,0,1,1,99,0,480


In [ ]:
%%time
for col in table.column_names():
    count_nulls = table[ table[col] == None ]
    print('amount of nulls for column {c} is {p}'.format(c = col, p =str(count_nulls.shape) ))
    table = table.fillna(col, 0)

In [44]:
table_epi.shape

(871112, 8)

In [47]:
table.shape

(3028657, 14)

## pensar en mejores consistency checks

In [48]:
%%time
unique_users = set(table['USER'].unique())
all_users = set(table['USER'])
len(unique_users), len(all_users)

CPU times: user 28.7 s, sys: 3.58 s, total: 32.3 s
Wall time: 15.3 s


In [49]:
print(unique_users == all_users) 
print(len(all_users))

True
3028657


In [50]:
table.sample(0.0001)

USER,TimeWeekEnd,CallsWeekDay,CallsWeekNight,TimeWeekDay,CallsWeekEnd,TimeWeekNight
384F696D7342544C7368567078376E457366484778673D3D ...,8036,39,12,8813,21,2960
634C68446D6A4968636965556230356D36587A4475413D3D ...,0,1,0,16,0,0
47626E546C7151696B7670794B623776736C4A4A68773D3D ...,3554,27,6,2263,5,4679
6F48724C457661426F502F586E68744E536B415932673D3D ...,452,50,1,2097,6,23
5679774732593169636B45522F6F31462B596B547A413D3D ...,0,0,0,0,0,0
356B63303157574E6A594A51587966463343343156773D3D ...,0,0,0,0,0,0
59624F6F50736E687A4F526463515337704F6E3367513D3D ...,0,0,0,0,0,0
6E45666E64534869747872646A4B327A4437717979773D3D ...,0,0,0,0,0,0
31644F74576A756E54613678354475636C65726D6A673D3D ...,0,0,0,0,0,0
473835532B4A545630547756383569435751377950413D3D ...,0,0,0,0,0,0


In [51]:
table.head()

USER,TimeWeekEnd,CallsWeekDay,CallsWeekNight,TimeWeekDay,CallsWeekEnd,TimeWeekNight
36766C5262716643446A53514F7246534E63664278673D3D ...,653,149,38,7470,23,1476
2F63486368745777513669346F364B363732694349673D3D ...,530,14,9,461,12,383
655A7341594B59396A5942617937645044714E7559513D3D ...,292,6,9,199,7,237
4F4A4A6D65304F3471544E7174354D464941707948773D3D ...,0,1,0,49,0,0
4345776E53674439427A3639342F54666B66693530673D3D ...,171,11,3,445,6,151
35584A656A396B516F7337365163326A62412B5A6D413D3D ...,9,11,2,64,2,8
4B494D415637543555316F307A324D42427976442B673D3D ...,943,50,12,6078,21,547
36367A4D7846664331413462396B5A686235704449773D3D ...,0,28,46,44,0,572
453436512F4C3338697031624876634A794F393472513D3D ...,1752,54,20,2813,27,1358
7976525842754576683568475763706B7845364D5A673D3D ...,0,1,1,99,0,480


## salvamos version 2 antes de agregar per antenna

In [53]:
!stat $output_dir

  File: `/home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links2_g14tog19_direcO'
  Size: 4096      	Blocks: 8          IO Block: 4096   directory
Device: 831h/2097d	Inode: 68337671    Links: 2
Access: (0755/drwxr-xr-x)  Uid: ( 3001/    juan)   Gid: ( 3001/    juan)
Access: 2016-07-01 13:11:09.801385322 +0000
Modify: 2016-07-01 13:11:12.193385357 +0000
Change: 2016-07-01 13:11:12.193385357 +0000
 Birth: -


In [54]:
%%time
table.save(output_dir)
#table.save(output_dir)

CPU times: user 1.35 s, sys: 144 ms, total: 1.49 s
Wall time: 1.6 s


# final table aggregation

en este caso los datasets hay que juntarlos en uno solo agrupados por user para luego asignar antennas y despues contar vuln_users per antenna

En teoria, en el primera version del file sum_links, los users ya vienen filtrados (por eso el group_int en el filename)

Hacemos un sanity check de esto, agregamos las antennas y agrupamos por antenna p/tener el primer _out file 

# FUCK YOU $\tiny{...\ and \ good\  night}$

In [33]:
month = 1
#grupo a analizar desde
g1 = 14
#grupo a analizar hasta
g2 = 19
groups = range(g1,g2+1)
direction = 'O'
load_dir = "output/2012/%s/" % month
!ls $load_dir
month,g1,g2,direction

antenna_usage_aggregation.csv  sum_links__group13_1  sum_links__group2_1
sum_links2_g0tog4_direcO       sum_links__group14_1  sum_links__group3_1
sum_links2_g14tog19_direcO     sum_links__group15_1  sum_links__group4_1
sum_links2_g5tog13_direcO      sum_links__group16_1  sum_links__group5_1
sum_links__group0_1	       sum_links__group17_1  sum_links__group6_1
sum_links__group10_1	       sum_links__group18_1  sum_links__group7_1
sum_links__group11_1	       sum_links__group19_1  sum_links__group8_1
sum_links__group12_1	       sum_links__group1_1   sum_links__group9_1


(1, 14, 19, 'O')

In [22]:
%%time
##read homeantenna_data
table =  gl.SFrame()

output_dir3 = 'output/2012/1/sum_links3'

antenna_output = 'output/2012/1/antenna_usage_aggregation.csv'
#antenna_output

manual_dir = 'output/2012/1/sum_links2_g0tog4_direcO/'
table1 = gl.load_sframe(manual_dir)

manual_dir = 'output/2012/1/sum_links2_g5tog13_direcO/'
#table = table.append(gl.load_sframe(manual_dir))
table2 = gl.load_sframe(manual_dir)

manual_dir = 'output/2012/1/sum_links2_g14tog19_direcO/'
table3 = gl.load_sframe(manual_dir)

#table = table.append(gl.load_sframe(manual_dir))

print(table.shape)

(0, 0)
CPU times: user 4 ms, sys: 12 ms, total: 16 ms
Wall time: 16.4 ms


In [23]:
table1.shape, table2.shape, table3.shape

((2523947, 14), (4540898, 14), (3028657, 14))

## consistency check
que en c/ tabla haya distintos hash(USER) % passes (o el valor del parametro del script que creaba el SumLinks)

In [24]:
%%time
all_tables = [table1,table2,table3]
control_col = 'hash_control'
passes = 20
for table in all_tables:
    table[control_col] = table['USER'].apply(lambda x: hash(x) % passes )
    
#table.groupby('type', gl.aggregate.COUNT)

CPU times: user 12 ms, sys: 32 ms, total: 44 ms
Wall time: 29.3 ms


In [25]:
%%time
for table in all_tables:
    print(table.groupby(control_col, gl.aggregate.COUNT)[control_col])

[4, 1, 3, 0, 2]
[10, 11, 9, 5, 7, 8, 12, 6, 13]
[14, 19, 18, 15, 17, 16]
CPU times: user 35 s, sys: 6.01 s, total: 41 s
Wall time: 7.02 s


# Results
yay

In [26]:
table = gl.SFrame()

for tab in all_tables:
    table = table.append(tab)

In [27]:
table.head()

USER,TimeWeekEnd,CallsWeekDay,CallsWeekNight,TimeWeekDay,CallsWeekEnd,TimeWeekNight
554E6E2F4956616774546E54416B62387271547970773D3D ...,3060,0,4,0,3,601
49436870653850724478704C6877726B716544684C673D3D ...,381,5,1,39,2,64
6E71613734743949502F4F784F4572346D72595959673D3D ...,25,5,8,272,4,704
432F2B3554785564347956623364576C5274705358773D3D ...,0,1,0,55,0,0
3376632F7830576A34775775327A32336231304C79513D3D ...,793,29,8,1172,21,331
5A61425144765A324A634949535668323944416A66513D3D ...,554,12,12,495,13,3548
782B4B30416C324A55314746516E5744704458306B673D3D ...,660,0,0,0,1,0
68396959373350324E726D722F644A494837735430773D3D ...,0,1,0,1,0,0
684446417944783548745650636E6B776B4C496E48413D3D ...,1189,16,3,778,6,192
52305952724B765A6A516A4B30554E6F2B4E46764E673D3D ...,14,17,2,379,1,29


In [28]:
table = table.remove_column('hash_control')

In [29]:
aggregator_dict = dict()
for col in table.column_names():
    if col == 'USER':
        continue
    aggregator_dict[col] = gl.aggregate.SUM(col)
aggregator_dict

{'CallsWeekDay': ('__builtin__sum__', ['CallsWeekDay']),
 'CallsWeekDay_EPI': ('__builtin__sum__', ['CallsWeekDay_EPI']),
 'CallsWeekEnd': ('__builtin__sum__', ['CallsWeekEnd']),
 'CallsWeekEnd_EPI': ('__builtin__sum__', ['CallsWeekEnd_EPI']),
 'CallsWeekNight': ('__builtin__sum__', ['CallsWeekNight']),
 'CallsWeekNight_EPI': ('__builtin__sum__', ['CallsWeekNight_EPI']),
 'TimeWeekDay': ('__builtin__sum__', ['TimeWeekDay']),
 'TimeWeekDay_EPI': ('__builtin__sum__', ['TimeWeekDay_EPI']),
 'TimeWeekEnd': ('__builtin__sum__', ['TimeWeekEnd']),
 'TimeWeekEnd_EPI': ('__builtin__sum__', ['TimeWeekEnd_EPI']),
 'TimeWeekNight': ('__builtin__sum__', ['TimeWeekNight']),
 'TimeWeekNight_EPI': ('__builtin__sum__', ['TimeWeekNight_EPI']),
 'VULNERABLE': ('__builtin__sum__', ['VULNERABLE'])}

In [30]:
%%time
import graphlab.aggregate as agg
#igualmente agrupo por las dudas
print(table.shape)
table = table.groupby('USER', aggregator_dict)
print(table.shape)

(10093502, 14)
(10093502, 14)
CPU times: user 4min 15s, sys: 38.5 s, total: 4min 54s
Wall time: 1min 16s


In [34]:
table.head()

USER,TimeWeekEnd_EPI,TimeWeekEnd,CallsWeekDay,CallsWeekNight,TimeWeekNight_EPI,CallsWeekEnd_EPI
794832576F74324554496A6B756933704E4D304566773D3D ...,0,7303,8,41,0,0
5763456C75454C6A2F7456706B774A4F6A55425964773D3D ...,0,733,9,5,0,0
36766C5262716643446A53514F7246534E63664278673D3D ...,315,653,149,38,2154,12
2F63486368745777513669346F364B363732694349673D3D ...,0,530,14,9,0,0
655A7341594B59396A5942617937645044714E7559513D3D ...,0,292,6,9,0,0
4F4A4A6D65304F3471544E7174354D464941707948773D3D ...,0,0,1,0,0,0
6E6E4C516949485463533865527665643835313142513D3D ...,60,0,0,0,40,1
65784D647A30565462454B55332B426E3353436537773D3D ...,0,0,6,2,0,0
43566E6D6243394E5662347763422F53515042555A773D3D ...,0,0,0,1,0,0
4345776E53674439427A3639342F54666B66693530673D3D ...,0,171,11,3,0,0


### note that approx we have 10M users in this month

In [36]:
output_dir3

'output/2012/1/sum_links3'

In [37]:
#re nuemramos VULNERABLE en caso de que se hayan acumulado valores cuando hicimos el groupby user agg SUM
table['VULNERABLE'] = table['VULNERABLE'] >0
#salvamos
table.save(output_dir3)


In [39]:
!du -sh $output_dir3

512M	output/2012/1/sum_links3


## LOAD ONCE AGAIN

In [13]:
antenna_usage =gl.SFrame.read_csv(antenna_output, delimiter='|') 

Finished parsing file /home/juan/mobility-study/argentina-scripts/output/2012/1/antenna_usage_aggregation.csv

Parsing completed. Parsed 100 lines in 0.0575 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int,int,int,int,int,int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/juan/mobility-study/argentina-scripts/output/2012/1/antenna_usage_aggregation.csv

Parsing completed. Parsed 4466 lines in 0.042842 secs.

In [98]:
perc = 0.99
col = 'USER'

antenna_usage[col].sketch_summary()

approx_sketch = antenna_usage[col].sketch_summary()
quant = approx_sketch.quantile(perc)
print(approx_sketch)

antenna_usage[antenna_usage[col]<quant][col].show()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |      4466     |   Yes    |
|        Min         |      1.0      |   Yes    |
|        Max         |    477068.0   |   Yes    |
|        Mean        | 2107.86654725 |   Yes    |
|        Sum         |   9413732.0   |   Yes    |
|      Variance      | 62765254.2227 |   Yes    |
| Standard Deviation | 7922.45253837 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |      2564     |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+----+----+----+----+----+----+----+----+----+----+
| value | 1  | 3  | 4  | 5  | 11 | 10 | 28 | 36 | 2  | 13 |
+-------+----+----+----+----+----+----+----+----+----+----+
| count | 21 | 16 | 16 | 15 | 14 | 13 | 12 | 12 | 11 | 11 |
+-------+----+----+----+----+----+----+----+----+----+----+

Quantiles: 
+-----+-----+-

In [16]:
call_columns = [col for col in antenna_usage.column_names() if "Call" in col and not("EPI" in col)]
epi_call_columns = [col for col in antenna_usage.column_names() if "Call" in col and("EPI" in col)]
#call_columns, epi_call_columns
antenna_usage['calls']= antenna_usage[call_columns]\
            .apply(lambda x: x[call_columns[0]]+ x[call_columns[1]]+ x[call_columns[2]] )
antenna_usage['vuln_calls']= antenna_usage[epi_call_columns]\
            .apply(lambda x: x[epi_call_columns[0]]+ x[epi_call_columns[1]]+ x[epi_call_columns[2]] )
antenna_usage.rename({'VULNERABLE':'vuln_users',''})

In [17]:
antenna_usage.head()

ANTENNA_ID_WEEKNIGHT_0,TimeWeekEnd_EPI,TimeWeekEnd,CallsWeekDay_EPI,CallsWeekDay,CallsWeekNight,TimeWeekNight_EPI
C1501,68347,2656384,1642,69769,22281,86649
BA629,7609,530932,181,14272,5798,13345
C2352,2761,142247,38,2561,788,2029
CO609,324033,2609634,12123,85016,28875,382065
CF565,0,1749,2,126,17,0
SF307,38665,2552385,711,66235,26876,44545
C1752,63155,1530147,756,30461,12278,66397
SL046,458074,289295,8651,4894,2749,585900
SF984,49718,3943738,2038,126541,41433,45361
C1322,18145,1142887,238,26116,10153,24142


In [18]:
perc = 0.99
col = 'calls'

antenna_usage[col].sketch_summary()

approx_sketch = antenna_usage[col].sketch_summary()
quant = approx_sketch.quantile(perc)
print(approx_sketch)

antenna_usage[antenna_usage[col]<quant][col].show()


+--------------------+-------------------+----------+
|        item        |       value       | is exact |
+--------------------+-------------------+----------+
|       Length       |        4466       |   Yes    |
|        Min         |        0.0        |   Yes    |
|        Max         |     20876596.0    |   Yes    |
|        Mean        |   109768.642857   |   Yes    |
|        Sum         |    490226759.0    |   Yes    |
|      Variance      | 1.30605894118e+11 |   Yes    |
| Standard Deviation |   361394.374773   |   Yes    |
|  # Missing Values  |         0         |   Yes    |
|  # unique values   |        4328       |    No    |
+--------------------+-------------------+----------+

Most frequent items:
+-------+----+---+------+----+---+-----+-----+----+------+-----+
| value | 24 | 2 | 1371 | 58 | 1 | 634 | 198 | 25 | 4677 | 161 |
+-------+----+---+------+----+---+-----+-----+----+------+-----+
| count | 5  | 4 |  3   | 3  | 3 |  3  |  3  | 3  |  3   |  3  |
+-------+----+-

In [19]:
perc = 0.95
col = 'vuln_calls'

antenna_usage[col].sketch_summary()

approx_sketch = antenna_usage[col].sketch_summary()
quant = approx_sketch.quantile(perc)
print(approx_sketch)

antenna_usage[antenna_usage[col]<quant][col].show()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |      4466     |   Yes    |
|        Min         |      0.0      |   Yes    |
|        Max         |   2982549.0   |   Yes    |
|        Mean        | 20318.7454098 |   Yes    |
|        Sum         |   90743517.0  |   Yes    |
|      Variance      | 6666580349.45 |   Yes    |
| Standard Deviation | 81649.1295082 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |      3038     |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+-----+----+----+----+----+----+----+----+----+----+
| value |  0  | 1  | 2  | 3  | 4  | 12 | 7  | 14 | 63 | 5  |
+-------+-----+----+----+----+----+----+----+----+----+----+
| count | 105 | 29 | 22 | 20 | 16 | 15 | 13 | 11 | 11 | 11 |
+-------+-----+----+----+----+----+----+----+----+----+----+

Quantiles: 
+-----+--

In [ ]:
perc = 0.95
col = 'calls'
vuln_col = "vuln_" + 

antenna_usage[col].sketch_summary()

approx_sketch = antenna_usage[col].sketch_summary()
quant = approx_sketch.quantile(perc)
print(approx_sketch)

antenna_usage[antenna_usage[col]<quant][col].show()


a = table_arg[table_arg[col] < table_arg[col].quantile(quant)][col]
b = table_arg[table_arg[col] < table_arg[col].quantile(quant)][vuln_col]

(b/a).hist(bins=30, normed = True, 
                  color = color_palette[5], alpha = 0.5, lw = 0)